In [1]:
!pip install transformers
!pip install textattack
# !pip install fasttext
!pip install spacy
!pip install pyspellchecker
!pip install textblob

     |████████████████████████████████| 1.9MB 19.1MB/s 
     |████████████████████████████████| 3.2MB 41.2MB/s 
     |████████████████████████████████| 890kB 55.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=dd76d3d3c3835c28e1e68d519dc07c75b6b4e14e2d86acfc722f8d99607c13a0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 358kB 17.6MB/s 
     |████████████████████████████████| 194kB 17.3MB/s 
     |████████████████████████████████| 778kB 44.5MB/s 
     |████████████████████████████████| 61kB 11.1MB/s 
     |████████████████████████████████| 20.1MB 167kB/s 
     |████████████████████████████████| 337kB 52.4MB/s 
     |████████████████████████████████| 102kB 16.2MB/s 
     |████████████████████████████████| 245kB 59.9MB/s 
     |████████████████████████████████| 112kB 59.4MB/s 
     |█████████████████████████

     |████████████████████████████████| 2.5MB 17.7MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/My Drive/Adversarial')

**scRNN**

In [4]:
"""This module provides the possibility to test a set randoms words"""
import json
import numpy as np
from keras.models import load_model
from robsut_wrod_reocginiton.binarize import noise_char, jumble_char

# 1 pre process steps

f = open('/content/drive/My Drive/Adversarial/robsut_wrod_reocginiton/vocabID.json','r')
data = json.loads(f.read())
f.close()
vocab = data['voc']
id2vocab = {int(k):v for k,v in data['idvoc'].items()}

def decode_word(X, calc_argmax):
    if calc_argmax:
        X = X.argmax(axis=-1)
    return ' '.join(id2vocab[x] for x in X)


def vectorize_data(vec_cleaned, alph, noise_type = 'JUMBLE', jumble_type='INT'):
    X_vec = np.zeros((int(len(vec_cleaned)),  len(alph) * 3), dtype=np.bool)
    for i, word in enumerate(vec_cleaned):
        if jumble_type == 'NO':
            x_feat, _ = noise_char(word, noise_type, alph)
        else:
            x_feat, _ = jumble_char(word, jumble_type, alph)
        X_vec[i] = x_feat
        # print(X_vec[i].shape)
    return X_vec.reshape((1,len(vec_cleaned),228))

alph = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz.,:;'*!?`$%&(){}[]-/\@_#"

# 2 test_set and featurization; you can provide a set of 20 words and only 20 words



# 3 the predictions

model_file = '/content/drive/My Drive/Adversarial/robsut_wrod_reocginiton/models/train_j-INT_n-JUMBLE_u-650_batch-20_ep-10_model.h5'
scrnn_model = load_model(model_file)




In [5]:
# test_set = ['Otiher', 'brokeryage','fidrms','inclfuding','Merrfill','Lfynch', 'potfential', 'stratfegies', 'mafrket',
#             'Frifday', 'afterfnoon', 'invefstment', 'editfions','newsfpapers', 'negowtiated',
#             'arranhgement','Ciable', 'Networok', 'undler', 'agrtee', 'excellent', 'goood'
import re

test_set = 'youu vvery bbad good fuckk sallam mam ,  "excelllend offerr'
test_set = "starts slowly , but adrien brody  in the title role  hleps make the film's conclusion poewrful and satifsying ."

def preprocess_input(data):
  data = re.sub('["\x96\x85½]', '', data)
  data = re.sub('ó', 'o', data)
  data = re.sub('[ãá]', 'a', data)
  data = re.sub('ç', 'c', data)
  data = re.sub('é', 'e', data)
  data = re.sub('ú', 'u', data)
  data = re.sub('É', 'e', data)
  data = re.sub('ï', 'i', data)
  
  d = data.split(' ')
  d_len = len(d)
  if len(d) < 20:
    tmp = len(d)
    for i in range(20 - len(d)):
      d.append('OOV')
    return [d], d_len


  elif len(d) > 20:
    i = 20
    results = []
    while i < len(d):
      results.append(d[i-20:i])
      i += 20
    tmp = d[i-20:]
    for itr in range(20 - len(tmp)):
      tmp.append('OOV')
    results.append(tmp)
    return results, d_len

  elif len(d) == 20:
    return [d], d_len

def scRNN_defense(data):
  inputs, length = preprocess_input(data)

  results = []
  for d in inputs:
    print(d)
    X_test = vectorize_data(d, alph)

    preds = np.argmax(scrnn_model.predict(X_test), axis=-1)
    pred_j = decode_word(preds[0], calc_argmax=False)

    results = results + pred_j.split()
    
  return ' '.join(results[:length])


scRNN_defense(test_set)

['starts', 'slowly', ',', 'but', 'adrien', 'brody', '', 'in', 'the', 'title', 'role', '', 'hleps', 'make', 'the', "film's", 'conclusion', 'poewrful', 'and', 'satifsying']
['.', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV']


'starts slowly , but children body a in the title role a helps make the firms conclusion powerful and starting .'

In [6]:
from Code.defense.generate_typos import GenerateTypos
from Code.defense.preprocess import Preprocess
from Code.defense.main_defense import MainDefense

In [24]:
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-ag-news")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Create the goal function using the model
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model_wrapper)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [25]:
# Import the dataset
from textattack.datasets import HuggingFaceDataset
dataset_train = HuggingFaceDataset("ag_news", None, "train")
# dataset_val = HuggingFaceDataset("rotten_tomatoes", None, "validation")
dataset_test = HuggingFaceDataset("ag_news", None, "test")

Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
textattack: Loading datasets dataset ag_news, split train.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
textattack: Loading datasets dataset ag_news, split test.


In [27]:
len(test_adv_examples)

1000

**Saving adversarials**

In [40]:
import pickle

with open('/content/drive/My Drive/Adversarial/Dataset/ag/ag_bert_insertion', 'rb') as handle:
    test_adv_examples = pickle.load(handle)


In [29]:
indices_list = []
itr = 0
test_labels = []
max = 0
for i in range(10000):
  try:
    text_input, ground_truth_output = dataset_test[i]
    # print()
    if len(text_input['text'].split(' ')) > max:
      max = len(text_input['text'].split(' '))
    # print(ground_truth_output)
    test_labels.append(ground_truth_output)
    # print('\n')
  except:
    print("An exception occurred")
    break
# if len(text_input['text'].split(' ')) < 500:
#   indices_list.append(i)
  itr += 1
# print(len(indices_list))
print(itr)
print(len(test_labels))
max

An exception occurred
7600
7600


149

In [30]:
class_0_indices = [i for i in range(len(test_labels)) if test_labels[i] == 0]
class_1_indices = [i for i in range(len(test_labels)) if test_labels[i] == 1]
class_2_indices = [i for i in range(len(test_labels)) if test_labels[i] == 2]
class_3_indices = [i for i in range(len(test_labels)) if test_labels[i] == 3]

indices_list = class_0_indices[:250] + class_1_indices[:250] + class_2_indices[:250] + class_3_indices[:250] 
len(indices_list)

1000

**Run our approach**

In [41]:
import torch
import time 
device = torch.device("cuda")
true_defenses = 0
cntr = 0
defended = []

for i, adv_example in zip(sorted(indices_list), test_adv_examples): 

  prt_txt = adv_example.perturbed_text()

  # Pass the prt_text to the defense approach

  try:
    t1 = time.time()
    # fixed_txt = Defense.check_input(prt_txt)
    fixed_txt = scRNN_defense(prt_txt)
    t2 = time.time()
    print('Time taken for this defense: {} secs'.format(t2 - t1))

    # Check the output of Defense
    org_txt = adv_example.original_text()
    # fixed_txt = prt_txt
    text_input, ground_truth_output = dataset_test[i]

    fixed_adv = tokenizer.encode(fixed_txt)
    # org_input = tokenizer.encode(org_txt)

    fixed_logits = model(input_ids=torch.tensor([fixed_adv['input_ids']]).to(device), attention_mask=torch.tensor([fixed_adv['attention_mask']]).to(device))
    
    # AG NEWS
    if torch.argmax(fixed_logits[0][0]) == ground_truth_output:
      true_defenses += 1
    else:
      print('******** Unsuccessful Defense **********')


    ## TOMATO - IMDB
    # if torch.sign(fixed_logits.logits[0][1] - fixed_logits.logits[0][0]) > 0:
    #   if ground_truth_output == 1:
    #     true_defenses += 1
    # elif ground_truth_output == 0:
    #   true_defenses += 1
    # else:
    #   print('******** Unsuccessful Defense **********')

  except:
    continue
  
  
  # print(torch.sign(fixed_logits.logits[0][1] - fixed_logits.logits[0][0]))
  print('Total number of adversarials so far: {}'.format(cntr + 1))
  print('Total number of successful defenses so far: {}'.format(true_defenses))
  cntr += 1
  # with open('/content/drive/My Drive/Adversarial/Dataset/ag/ag_bert_insertion_defended', 'wb') as handle:
  #   pickle.dump(defended, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print('\n')


print('\n')
print('Acc: {}'.format(true_defenses / len(adv_examples)))

Streaming output truncated to the last 5000 lines.
Time taken for this defense: 0.09057998657226562 secs
******** Unsuccessful Defense **********
Total number of adversarials so far: 340
Total number of successful defenses so far: 201


['Jerkens', 'makes', 'right', 'call', 'with', 'Society', 'Sfelection', 'Trainrer', 'Allen', 'Jerktens', 'hemmed', 'and', 'hawed', 'this', 'past', 'week', 'over', 'running', 'Society', 'Soelection']
['in', 'Satturday', '#39;s', 'Grade', '1', 'Alabama', 'at', 'Saratoga.', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV', 'OOV']
Time taken for this defense: 0.09372472763061523 secs
******** Unsuccessful Defense **********
Total number of adversarials so far: 341
Total number of successful defenses so far: 201


['Unknown', 'Nesterenko', 'Makes', 'World', 'Headlines', '(Reuters)', 'Remuters', '-', "Belarus'", 'Yuliya', 'Nesterenko', 'won', 'the', "top\\women's", 'athletics', 'gold', 'medal', 'at', 'the', 'Olympics']
['on', 'Saturd

NameError: ignored

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

from IPython.core.display import display, HTML
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

,original_text,perturbed_text
0,"lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .","lvoingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweteness largely wtihout stickiness ."
1,consistently clever and suspenseful .,consistently clveer and suspenesful .
2,"it's like a "" big chill "" reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .","it's like a "" big chill "" reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists ."
3,"the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .","the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with treemndous sklil ."
4,"red dragon "" never cuts corners .","red dragon "" never ctus corners ."
5,fresnadillo has something serious to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .,fresnadillo has sometihng seirous to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .
6,throws in enough clever and unexpected twists to make the formula feel fresh .,throws in enough celver and unexpected twists to make the formula feel fersh .
7,weighty and ponderous but every bit as filling as the treat of the title .,weighty and ponderous but every bit as fliling as the treat of the title .
8,"a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .","a real audiencep-leaser that will strike a chrod with anynoe who's eevr waited in a doctor's office , emergency room , hospital bed or insurance company office ."
9,generates an enormous feeling of empathy for its characters .,generates an enomrous feeilng of empathy for its characters .
